In [1]:
# install dependencies

import sys
!{sys.executable} -m pip install pandas icd matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
# set relative directories

home_dir = "/Users/lhuang21/Documents/Programming/MD+_Datathon_2023/"
data_file_name = "initial_features_w_hadm_id_raw.csv"

In [3]:
import pandas as pd
import os.path
import icd
import matplotlib.pyplot as plt
from collections import defaultdict

# set up max rows and columns limit to 100
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option("display.width", 200)

In [4]:
# read in csv
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv(os.path.join(home_dir, data_file_name))

/var/folders/65/qxp2fcwj00bff0rdg3hmhyx80000gn/T/ipykernel_24165/2097460905.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(home_dir, data_file_name))


In [5]:
# show df shape
print(f"df shape: {df.shape}")

# show column data types
print(df.dtypes)

# show data preview
print(df.head(30), end="\n\n")

# show number of rows
print(f"{len(df)} rows")

# show number of unique subject_id's
print(f"{len(df['subject_id'].unique())} unique subject_id's")

# show number of unique icd_codes
print(f"{len(df['icd_code'].unique())} unique icd_codes")

df shape: (38924112, 11)
subject_id              int64
hadm_id                 int64
age                     int64
gender                 object
max_bmi               float64
max_blood_pressure     object
icd_code               object
icd_version             int64
race                   object
insurance              object
language               object
dtype: object
    subject_id   hadm_id  age gender  max_bmi max_blood_pressure icd_code  icd_version   race insurance language
0     10010399  25356745   54      F     28.0              92/66  3051               9  WHITE  Medicaid  ENGLISH
1     10010399  25356745   54      F     28.0              92/66  V103               9  WHITE  Medicaid  ENGLISH
2     10010399  25356745   54      F     28.0              92/66  V153               9  WHITE  Medicaid  ENGLISH
3     10010399  25356745   54      F     28.0              92/66  138                9  WHITE  Medicaid  ENGLISH
4     10010399  25356745   54      F     28.0              92/66  

In [6]:
# find if any subjects have:
# - icd_version "9" / associated ICD9 codes for CKD
# - icd_version "10" / associated ICD10 codes for CKD
# and if so, save to variables
ICD9_CKD_PREFIXES = ["585", "28521", "4039"]
df_ckd_icd9 = df[(df['icd_version'] == 9) & (
    df['icd_code'].str.startswith(tuple(ICD9_CKD_PREFIXES)))]

ICD10_CKD_PREFIXES = ["I12", "N18", "N19", "N990", "I130", "O102", "E1122", "D631", ]
df_ckd_icd10 = df[(df['icd_version'] == 10) &
                  df['icd_code'].str.startswith(tuple(ICD10_CKD_PREFIXES))]

# show number of unique subject_id's with icd_version "9" / icd_code "585"
icd9_ckd_subject_ids = df_ckd_icd9['subject_id'].unique()
print(f"{len(icd9_ckd_subject_ids)} unique subject_id's with icd_version '9' / icd_code '585'")

# show number of unique subject_id's with icd_version "10" / icd_code that starts with "N18"
icd10_ckd_subject_ids = df_ckd_icd10['subject_id'].unique()
print(f"{len(icd10_ckd_subject_ids)} unique subject_id's with icd_version '10' / icd_code that starts with 'N18'")

total_num_ckd_subject_ids = len(
    icd9_ckd_subject_ids) + len(icd10_ckd_subject_ids)
print(f"{total_num_ckd_subject_ids} unique subject_id's with icd_version '9' / icd_code '585' or icd_version '10' / icd_code that starts with 'N18'")

# get a dataframe that has all the subject_id's from df_ckd_icd9 and df_ckd_icd10
df_ckd_icds_only = pd.concat([df_ckd_icd9, df_ckd_icd10])

# get number of unique subject ids in df_ckd_only
print(
    f"{len(df_ckd_icds_only['subject_id'].unique())} unique subject_id's in df_ckd_only")
print(f"This means there's {total_num_ckd_subject_ids - len(df_ckd_icds_only['subject_id'].unique())} patient with both ICD9 and ICD10 codes")

# show preview
print(df_ckd_icds_only.head())


14096 unique subject_id's with icd_version '9' / icd_code '585'
11495 unique subject_id's with icd_version '10' / icd_code that starts with 'N18'
25591 unique subject_id's with icd_version '9' / icd_code '585' or icd_version '10' / icd_code that starts with 'N18'
22576 unique subject_id's in df_ckd_only
This means there's 3015 patient with both ICD9 and ICD10 codes
      subject_id   hadm_id  age gender  max_bmi max_blood_pressure icd_code  icd_version   race insurance language
1275    10132888  21293774   89      M      NaN                NaN  5853               9  ASIAN     Other        ?
1276    10132888  21293774   89      M      NaN                NaN  40390              9  ASIAN     Other        ?
2054    10277901  24572383   91      M      NaN                NaN  5859               9  OTHER  Medicare        ?
2055    10277901  24572383   91      M      NaN                NaN  40390              9  OTHER  Medicare        ?
2078    10277901  23194882   91      M      NaN          

In [7]:
# get all unique icd codes from df_ckd and convert to pd series
uniq_ckd_icd_codes = pd.Series(df_ckd_icds_only['icd_code'].unique())

# print num unique codes
print(f"{len(uniq_ckd_icd_codes)} unique icd_codes indicating CKD")

# print all unique codes
print(uniq_ckd_icd_codes)

# get counts of unique subject ids per unique CKD icd code
df_ckd_icd_counts = df_ckd_icds_only.groupby('icd_code')['subject_id'].nunique().sort_values(ascending=False)
print(df_ckd_icd_counts)

29 unique icd_codes indicating CKD
0     5853   
1     40390  
2     5859   
3     40391  
4     5856   
5     5854   
6     28521  
7     5855   
8     5851   
9     5852   
10    D631   
11    E1122  
12    I120   
13    N185   
14    N186   
15    N183   
16    I129   
17    N189   
18    I130   
19    N184   
20    N990   
21    N182   
22    N181   
23    N19    
24    O10211 
25    O1022  
26    O10213 
27    O10212 
28    O1023  
dtype: object
icd_code
40390      10634
5859        9032
I129        6320
N189        5846
E1122       4640
5853        4312
N183        4301
I130        3612
28521       2902
5856        2441
40391       2374
D631        2118
N186        1825
5854        1587
I120        1404
N184        1250
5852         827
N182         622
5855         475
N185         401
N990         166
5851          89
N181          73
N19           32
O10212         3
O1022          3
O10213         2
O10211         1
O1023          1
Name: subject_id, dtype: int64


In [8]:
# get all unique icd codes from df corresponding to subject_id's from df_ckd_only
uniq_icd_codes_ckd_subjects = df[df['subject_id'].isin(
    df_ckd_icds_only['subject_id'])]['icd_code'].unique()

# print num of unique icd codes amongst ckd patients
print(f"{len(uniq_icd_codes_ckd_subjects)} unique icd_codes amongst CKD subjects")

# extract non CKD ICD codes seen in CKD patients
non_ckd_icd_codes_from_ckd_pts = [
    icd_code for icd_code in uniq_icd_codes_ckd_subjects if icd_code not in uniq_ckd_icd_codes]

# get counts of unique ckd pts with each icd code sorted by frequency
icd_code_counts_ckd = df[df['subject_id'].isin(df_ckd_icds_only['subject_id'])].groupby(
    'icd_code')['subject_id'].nunique().sort_values(ascending=False)
# exclude icd codes that are not CKD
icd_code_counts_ckd = icd_code_counts_ckd[icd_code_counts_ckd.index.isin(
    non_ckd_icd_codes_from_ckd_pts)]

# pick non-CKD icd codes seen in at least 5% of CKD patients
CUTOFF_FREQUENCY = 0.05
selected_non_ckd_icd_codes_from_ckd_pts = icd_code_counts_ckd[
    icd_code_counts_ckd >= CUTOFF_FREQUENCY * total_num_ckd_subject_ids].index
# print counts of CKD pts associated with these picked out icd codes
print(f"\n{len(selected_non_ckd_icd_codes_from_ckd_pts)} unique non-CKD icd codes seen in at least {int(CUTOFF_FREQUENCY * 100)}% of ckd patients, counts below")
print(icd_code_counts_ckd[icd_code_counts_ckd.index.isin(
    selected_non_ckd_icd_codes_from_ckd_pts)])


15383 unique icd_codes amongst CKD subjects

144 unique non-CKD icd codes seen in at least 5% of ckd patients, counts below
icd_code
40390      10634
5859        9032
2724        8997
5849        8470
E785        7554
4019        6844
4280        6826
N179        6443
I129        6320
41401       6011
25000       5870
N189        5846
Z87891      5503
42731       5447
53081       5359
2859        5324
I2510       5078
V1582       4837
E1122       4640
K219        4550
5853        4312
N183        4301
5990        4189
V5861       3808
V5867       3790
I130        3612
Z794        3561
D649        3496
2720        3467
311         3442
V5866       3368
Z7901       3345
I10         3173
2767        3029
I4891       2969
Z66         2921
2449        2906
28521       2902
F329        2893
486         2871
2762        2869
412         2788
Y929        2713
E872        2697
2761        2686
41400       2683
D62         2634
Z7902       2630
V4986       2624
27651       2617
N390        2615


In [9]:
# get all rows for patients with ckd
df_ckd_all_original_features = df[df['subject_id'].isin(
    df_ckd_icds_only['subject_id'])]

# get patients not in the df_ckd_only dataframe
df_non_ckd_all_original_features = df[~df['subject_id'].isin(
    df_ckd_icds_only['subject_id'])]
# print total number of unique patients in df
print(f"{len(df['subject_id'].unique())} total unique subject_id's")

# print number of unique patients in df_ckd_only
print(
    f"{len(df_ckd_all_original_features['subject_id'].unique())} unique CKD subject_id's")

# print number of non ckd patients
print(
    f"{len(df_non_ckd_all_original_features['subject_id'].unique())} unique non-CKD subject_id's")

# get the number of unique non CKD subject_ids associated with each selected non CKD icd_code
icd_code_counts_non_ckd = df_non_ckd_all_original_features[df_non_ckd_all_original_features['icd_code'].isin(selected_non_ckd_icd_codes_from_ckd_pts)].groupby(
    'icd_code')['subject_id'].nunique().sort_values(ascending=False)
print(f"\n{len(icd_code_counts_non_ckd)} selected non-CKD icd codes and counts of non-CKD patients associated with them")
display(icd_code_counts_non_ckd)

# filter rows of ckd patients for only icd codes in selected_non_ckd_icd_codes_from_ckd_pts
df_ckd_selected_features_only = df_ckd_all_original_features[df_ckd_all_original_features['icd_code'].isin(
    selected_non_ckd_icd_codes_from_ckd_pts)].reset_index(drop=True)
print("ckd pts preview")
display(df_ckd_selected_features_only['icd_code'].unique().shape)
display(df_ckd_selected_features_only.head())

# filter rows of non-ckd patients for only icd codes in selected_non_ckd_icd_codes_from_ckd_pts
df_non_ckd_selected_features_only = df_non_ckd_all_original_features[df_non_ckd_all_original_features['icd_code'].isin(
    selected_non_ckd_icd_codes_from_ckd_pts)].reset_index(drop=True)
display(df_non_ckd_selected_features_only['icd_code'].unique().shape)
print("non ckd pts preview")
display(df_non_ckd_selected_features_only.head())


180640 total unique subject_id's
22576 unique CKD subject_id's
158064 unique non-CKD subject_id's

129 selected non-CKD icd codes and counts of non-CKD patients associated with them


icd_code
4019       42897
I10        28348
2724       24451
E785       20349
53081      18596
Z87891     15853
311        15774
K219       14517
F329       13583
25000      13531
V1582      13012
3051       12507
F419       12224
41401      11852
30000      11653
2859       11268
42731      11007
2449        9740
5990        9378
2720        9008
Y929        8993
5849        8825
49390       8559
I2510       8360
E119        8320
4280        7606
V5861       7343
E669        7023
N179        6951
E039        6842
27800       6789
D649        6743
V5866       6713
D62         6065
32723       6038
Z7902       5990
Z7901       5978
2851        5943
2761        5923
27651       5738
486         5676
I4891       5671
G4733       5548
N390        5489
Z66         5431
42789       5337
78659       5139
56400       5139
V4986       4990
33829       4949
496         4916
E8497       4735
V5867       4638
E871        4531
2762        4484
73300       4483
Z794        4436
412         4422
78791

ckd pts preview


(144,)

,subject_id,hadm_id,age,gender,max_bmi,max_blood_pressure,icd_code,icd_version,race,insurance,language
0,10132888,21293774,89,M,NaN,NaN,V1582,9,ASIAN,Other,?
1,10132888,21293774,89,M,NaN,NaN,42731,9,ASIAN,Other,?
2,10132888,21293774,89,M,NaN,NaN,E8497,9,ASIAN,Other,?
3,10132888,21293774,89,M,NaN,NaN,E8798,9,ASIAN,Other,?
4,10132888,21293774,89,M,NaN,NaN,60000,9,ASIAN,Other,?


(129,)

non ckd pts preview


,subject_id,hadm_id,age,gender,max_bmi,max_blood_pressure,icd_code,icd_version,race,insurance,language
0,10010399,25356745,54,F,28.0,92/66,3051,9,WHITE,Medicaid,ENGLISH
1,10010399,25356745,54,F,28.0,92/66,56400,9,WHITE,Medicaid,ENGLISH
2,10010399,23084254,54,F,28.0,92/66,4019,9,WHITE,Medicaid,ENGLISH
3,10010399,23084254,54,F,28.0,92/66,3051,9,WHITE,Medicaid,ENGLISH
4,10010399,25356745,54,F,28.0,92/66,3051,9,WHITE,Medicaid,ENGLISH


In [10]:
# add ckd statuses
def add_ckd_status_col(df, status: bool):
    # add a column of all True variables to df
    df['ckd_status'] = status
    return df

display(add_ckd_status_col(df_ckd_selected_features_only, True).head())
display(add_ckd_status_col(df_non_ckd_selected_features_only, False).head())

,subject_id,hadm_id,age,gender,max_bmi,max_blood_pressure,icd_code,icd_version,race,insurance,language,ckd_status
0,10132888,21293774,89,M,NaN,NaN,V1582,9,ASIAN,Other,?,True
1,10132888,21293774,89,M,NaN,NaN,42731,9,ASIAN,Other,?,True
2,10132888,21293774,89,M,NaN,NaN,E8497,9,ASIAN,Other,?,True
3,10132888,21293774,89,M,NaN,NaN,E8798,9,ASIAN,Other,?,True
4,10132888,21293774,89,M,NaN,NaN,60000,9,ASIAN,Other,?,True


,subject_id,hadm_id,age,gender,max_bmi,max_blood_pressure,icd_code,icd_version,race,insurance,language,ckd_status
0,10010399,25356745,54,F,28.0,92/66,3051,9,WHITE,Medicaid,ENGLISH,False
1,10010399,25356745,54,F,28.0,92/66,56400,9,WHITE,Medicaid,ENGLISH,False
2,10010399,23084254,54,F,28.0,92/66,4019,9,WHITE,Medicaid,ENGLISH,False
3,10010399,23084254,54,F,28.0,92/66,3051,9,WHITE,Medicaid,ENGLISH,False
4,10010399,25356745,54,F,28.0,92/66,3051,9,WHITE,Medicaid,ENGLISH,False


585* - CKD with stages

403.90  	 	
Hy kid NOS w cr kid I-IV
403.91  	 	
Hyp kid NOS w cr kid V

28521 - Anemia in chronic kidney disease (UNK)

I120 - Hyp chr kidney disease w stage 5 chr kidney disease or ESRD
I129 - Hypertensive chronic kidney disease w stg 1-4/unsp chr kidney

N18.1  	 	
Chronic kidney disease, stage 1
N18.2  	 	
Chronic kidney disease, stage 2 (mild)
N18.30	‑	N18.32  	 	N18.3 Chronic kidney disease, stage 3 (moderate)
N18.4  	 	
Chronic kidney disease, stage 4 (severe)
N18.5  	 	
Chronic kidney disease, stage 5
N18.6  	 	
End stage renal disease
N18.9  	 	
Chronic kidney disease, unspecified

N19 - Unspecified kidney failure

N99.0 - Postprocedural (acute) (chronic) kidney failure

I13.0 - Hypertensive heart and chronic kidney disease with heart failure and stage 1 through stage 4 chronic kidney disease, or unspecified chronic kidney disease

O10.211	‑	O10.219  	 	O10.21 Pre-existing hypertensive chronic kidney disease complicating pregnancy
O10.22  	 	
Pre-existing hyp chronic kidney disease comp childbirth
O10.23  	 	
Pre-existing hyp chronic kidney disease comp the puerperium

E11.22  Type 2 diabetes mellitus with diabetic chronic kidney disease


D63.1 - Anemia in chronic kidney disease

In [11]:

# stage 5 and esrd are basically the same
# any ambiguous ones - just leave it out
#   - stages I-IV: "40390", "I129", "I130",
#   - unknown: "N189", "28521", "N19", "N990", "O102", "E1122", "D631"

CKD_STAGE_TO_ICD = {
    "1": {"5851", "N181"},
    "2": {"5852", "N182"},
    "3": {"5853", "N183"},
    "4": {"5854", "N185", "N186"},
    "5": {"5855", "40391", "I120"},  # setting equivalent to ESRD
}

ICD_TO_CKD_STAGE = defaultdict()
for stage, icd_set in CKD_STAGE_TO_ICD.items():
    for icd in icd_set:
        ICD_TO_CKD_STAGE[icd] = stage
display(ICD_TO_CKD_STAGE)


def assign_ckd_stage_to_icd_code(icd_code):
    for key in ICD_TO_CKD_STAGE.keys():
        if icd_code.startswith(key):
            return ICD_TO_CKD_STAGE[key]
    return 

# find all patients in df_ckd_selected_features_only with ckd stage icd codes
df_ckd_selected_features_only['ckd_stage'] = df_ckd_selected_features_only['icd_code'].apply(
    assign_ckd_stage_to_icd_code)
df_non_ckd_selected_features_only['ckd_stage'] = 0

# concatenate subject_id and hadm_id as the new index
df_ckd_selected_features_only['subject_id_hadm_id'] = df_ckd_selected_features_only['subject_id'].astype(
    str) + "_" + df_ckd_selected_features_only['hadm_id'].astype(str)
df_non_ckd_selected_features_only['subject_id_hadm_id'] = df_non_ckd_selected_features_only['subject_id'].astype(
    str) + "_" + df_non_ckd_selected_features_only['hadm_id'].astype(str)

# set the new index
df_ckd_selected_features_only.set_index(
    'subject_id_hadm_id', inplace=True)
df_non_ckd_selected_features_only.set_index(
    'subject_id_hadm_id', inplace=True)

# preview new subjects
display(
    df_ckd_selected_features_only[~df_ckd_selected_features_only['ckd_stage'].isnull()].head())

display(df_non_ckd_selected_features_only.head())

defaultdict(None,
            {'N181': '1',
             '5851': '1',
             'N182': '2',
             '5852': '2',
             '5853': '3',
             'N183': '3',
             'N186': '4',
             'N185': '4',
             '5854': '4',
             '40391': '5',
             '5855': '5',
             'I120': '5'})

,subject_id,hadm_id,age,gender,max_bmi,max_blood_pressure,icd_code,icd_version,race,insurance,language,ckd_status,ckd_stage
subject_id_hadm_id,,,,,,,,,,,,,
10132888_21293774,10132888,21293774,89,M,NaN,NaN,5853,9,ASIAN,Other,?,True,3
10246901_22999738,10246901,22999738,51,M,29.5,99/65,I120,10,WHITE,Other,ENGLISH,True,5
10246901_22729115,10246901,22729115,51,M,29.5,99/65,I120,10,WHITE,Other,ENGLISH,True,5
10246901_22729115,10246901,22729115,51,M,29.5,99/65,N186,10,WHITE,Other,ENGLISH,True,4
10246901_22999738,10246901,22999738,51,M,29.5,99/65,I120,10,WHITE,Other,ENGLISH,True,5


,subject_id,hadm_id,age,gender,max_bmi,max_blood_pressure,icd_code,icd_version,race,insurance,language,ckd_status,ckd_stage
subject_id_hadm_id,,,,,,,,,,,,,
10010399_25356745,10010399,25356745,54,F,28.0,92/66,3051,9,WHITE,Medicaid,ENGLISH,False,0
10010399_25356745,10010399,25356745,54,F,28.0,92/66,56400,9,WHITE,Medicaid,ENGLISH,False,0
10010399_23084254,10010399,23084254,54,F,28.0,92/66,4019,9,WHITE,Medicaid,ENGLISH,False,0
10010399_23084254,10010399,23084254,54,F,28.0,92/66,3051,9,WHITE,Medicaid,ENGLISH,False,0
10010399_25356745,10010399,25356745,54,F,28.0,92/66,3051,9,WHITE,Medicaid,ENGLISH,False,0


In [15]:
def _one_hot_encode_icd_cols_only(df):
    '''
    Extract one hot encoded icd codes from a dataframe.
    '''
    # one hot encode by icd codes
    icd_cols_one_hot_encoded = pd.get_dummies(df, columns=['icd_code'])

    # group one-hot ICD and apply any() on the columns prefixed by icd_code to get subject_id : icd_code mapping as true / false vectors
    icd_vectorized_by_hadm_id = icd_cols_one_hot_encoded.groupby(
        'subject_id_hadm_id').any()

    # get only the columns that start with icd_code_
    icd_code_columns = icd_vectorized_by_hadm_id.columns.str.startswith(
        'icd_code_')
    w = icd_vectorized_by_hadm_id.loc[:, icd_code_columns]

    # display(w.head())
    # display(w.shape)

    return w


def _extract_demographic_data(df):
    '''string aggs selected columns and merges duplicates across other rows'''
    # columns to string_agg 'race', 'insurance', 'language' columns
    COLS_TO_STR_AGG = ['race', 'insurance', 'language']

    str_agged_rows = df.groupby('subject_id_hadm_id')[COLS_TO_STR_AGG].agg(
        '|'.join)

    return pd.concat([df.drop(columns=COLS_TO_STR_AGG).drop_duplicates(), str_agged_rows], axis=1)


def select_ckd_stage_from_list_of_stages(ckd_stage_list):
    # select the max non-null value
    ckd_stage_list = [x for x in ckd_stage_list if x is not None]
    if len(ckd_stage_list) == 0:
        return None
    return max(ckd_stage_list)


def convert_icd_cols_to_one_hot(df):
    '''
    Convert 'icd_col' columns to multiple one hot encoded icd codes while preserving demographic data.
    '''
    df = df.copy(deep=True)

    # get pt ICD columns one hot encoded
    icd_oh = _one_hot_encode_icd_cols_only(df)

    # discard icd_code and icd_version columns to get only demographic data
    hadm_id_to_ckd_stage = df.groupby('subject_id_hadm_id')['ckd_stage'].aggregate(
        select_ckd_stage_from_list_of_stages).to_frame()

    demographic_cols_only = df.drop(
        columns=['icd_code', 'icd_version', 'ckd_stage'])
    hadm_indexed = demographic_cols_only.reset_index(
    ).drop_duplicates().reset_index(drop=True).set_index('subject_id_hadm_id')

    d = pd.concat([_extract_demographic_data(hadm_indexed),
                  hadm_id_to_ckd_stage, icd_oh], axis=1)
    return d


# CKD
df_ckd_icd_one_hot_selected_features = convert_icd_cols_to_one_hot(
    df_ckd_selected_features_only)

# non CKD
df_non_ckd_icd_one_hot_selected_features = convert_icd_cols_to_one_hot(
    df_non_ckd_selected_features_only)

# merge into one dataset
df_one_hot_icd_selected_features = pd.concat(
    [df_ckd_icd_one_hot_selected_features, df_non_ckd_icd_one_hot_selected_features], axis=0)


In [41]:
# Perform additional preprocessing on the feature column names, extract BP to systolic and diastolic

display(df_one_hot_icd_selected_features.shape)
display(df_one_hot_icd_selected_features.head(10))


def extract_ckd_subset(df):
    return df[df['ckd_status']]


def extract_non_ckd_subset(df):
    return df[~df['ckd_status']]


def extract_blood_pressure_components(df):
    df = df.copy(deep=True)
    BP_COL_NAME = 'max_blood_pressure'
    # split max_blood_pressure column on "/" into two columns
    df[['max_blood_pressure_systolic', 'max_blood_pressure_diastolic']] = df[BP_COL_NAME].str.split(
        "/", expand=True)
    df.drop(BP_COL_NAME, axis=1, inplace=True)
    return df


def prepend_feature_columns(df):
    df = df.copy(deep=True)
    # add F_ to the feature columns
    FEATURE_COLUMNS = ['max_blood_pressure_systolic',
                       'max_blood_pressure_diastolic', 'age', 'gender',
                       'max_bmi'
                       ]
    # prepend "F_" to column name if column has a name in FEATURE_COLUMNS or starts with "icd_code_"
    df.columns = ['F_' + x if x in FEATURE_COLUMNS or x.startswith(
        'icd_code_') else x for x in df.columns]
    return df


df_final = extract_blood_pressure_components(
    df_one_hot_icd_selected_features)

df_final = prepend_feature_columns(df_final)

# print proportion of patients with ckd_status true
print(f"{len(extract_ckd_subset(df_final))} CKD observations")

# print proportion of patients in z with ckd_status true
print(f"{len(extract_non_ckd_subset(df_final))} non-CKD observations")

# save final dataset to csv
df_final.to_csv('ckd_icd_codes_ckd_status_stages_grouped_hadmid_v1.csv')


(365693, 155)

,subject_id,hadm_id,age,gender,max_bmi,max_blood_pressure,ckd_status,race,insurance,language,ckd_stage,icd_code_0389,icd_code_2449,icd_code_25000,icd_code_25040,icd_code_25060,icd_code_2720,icd_code_2724,icd_code_2749,icd_code_2760,icd_code_2761,icd_code_2762,icd_code_27651,icd_code_27652,icd_code_2767,icd_code_2768,icd_code_27800,icd_code_2809,icd_code_2851,icd_code_28521,icd_code_2859,icd_code_2875,icd_code_28860,icd_code_30000,icd_code_3051,icd_code_311,icd_code_32723,icd_code_33829,icd_code_3572,icd_code_4019,icd_code_40390,icd_code_40391,icd_code_41071,icd_code_412,icd_code_41400,icd_code_41401,icd_code_4168,icd_code_4240,icd_code_4241,icd_code_42731,icd_code_42789,icd_code_4280,icd_code_42822,icd_code_42823,icd_code_42832,icd_code_42833,icd_code_4439,icd_code_4589,icd_code_486,icd_code_49390,icd_code_496,icd_code_51881,icd_code_53081,icd_code_56400,icd_code_58381,icd_code_5845,icd_code_5849,icd_code_5853,icd_code_5854,icd_code_5856,icd_code_5859,icd_code_5990,icd_code_60000,icd_code_71590,icd_code_73300,icd_code_78659,icd_code_78791,icd_code_78820,icd_code_79092,icd_code_79902,icd_code_99592,icd_code_A419,icd_code_D509,icd_code_D62,icd_code_D631,icd_code_D649,icd_code_D696,icd_code_E039,icd_code_E1122,icd_code_E1165,icd_code_E119,icd_code_E669,icd_code_E785,icd_code_E8497,icd_code_E860,icd_code_E871,icd_code_E872,icd_code_E875,icd_code_E8798,icd_code_F329,icd_code_F419,icd_code_G4733,icd_code_G92,icd_code_I10,icd_code_I120,icd_code_I129,icd_code_I130,icd_code_I214,icd_code_I2510,icd_code_I252,icd_code_I480,icd_code_I4891,icd_code_I5022,icd_code_I5023,icd_code_I5032,icd_code_I5033,icd_code_I959,icd_code_J189,icd_code_J449,icd_code_J9601,icd_code_K219,icd_code_K5900,icd_code_M109,icd_code_N179,icd_code_N183,icd_code_N186,icd_code_N189,icd_code_N390,icd_code_N400,icd_code_R0902,icd_code_V1251,icd_code_V1254,icd_code_V1582,icd_code_V4501,icd_code_V4511,icd_code_V4581,icd_code_V4582,icd_code_V4986,icd_code_V5861,icd_code_V5866,icd_code_V5867,icd_code_Y92009,icd_code_Y92230,icd_code_Y92239,icd_code_Y929,icd_code_Z515,icd_code_Z66,icd_code_Z7901,icd_code_Z7902,icd_code_Z794,icd_code_Z8673,icd_code_Z87891,icd_code_Z951,icd_code_Z955,icd_code_Z992
subject_id_hadm_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10132888_21293774,10132888,21293774,89,M,NaN,NaN,True,ASIAN,Other,?,3,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
10246901_28798682,10246901,28798682,51,M,29.5,99/65,True,WHITE,Other,ENGLISH,None,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,Fal

99922 CKD observations
265771 non-CKD observations


In [45]:
# print df_final col names
print("\"\n\"".join(list(df_final.columns)))

subject_id"
"hadm_id"
"F_age"
"F_gender"
"F_max_bmi"
"ckd_status"
"race"
"insurance"
"language"
"ckd_stage"
"F_icd_code_0389   "
"F_icd_code_2449   "
"F_icd_code_25000  "
"F_icd_code_25040  "
"F_icd_code_25060  "
"F_icd_code_2720   "
"F_icd_code_2724   "
"F_icd_code_2749   "
"F_icd_code_2760   "
"F_icd_code_2761   "
"F_icd_code_2762   "
"F_icd_code_27651  "
"F_icd_code_27652  "
"F_icd_code_2767   "
"F_icd_code_2768   "
"F_icd_code_27800  "
"F_icd_code_2809   "
"F_icd_code_2851   "
"F_icd_code_28521  "
"F_icd_code_2859   "
"F_icd_code_2875   "
"F_icd_code_28860  "
"F_icd_code_30000  "
"F_icd_code_3051   "
"F_icd_code_311    "
"F_icd_code_32723  "
"F_icd_code_33829  "
"F_icd_code_3572   "
"F_icd_code_4019   "
"F_icd_code_40390  "
"F_icd_code_40391  "
"F_icd_code_41071  "
"F_icd_code_412    "
"F_icd_code_41400  "
"F_icd_code_41401  "
"F_icd_code_4168   "
"F_icd_code_4240   "
"F_icd_code_4241   "
"F_icd_code_42731  "
"F_icd_code_42789  "
"F_icd_code_4280   "
"F_icd_code_42822  "
"F_icd_cod

In [65]:
# Remove features associated directly with CKD and dialysis to prevent data leakage

DATA_FILE = 'ckd_icd_codes_ckd_status_stages_grouped_hadmid_v1.csv'

# read in ckd_icd_codes_ckd_status_stages_grouped_hadmid_v1.csv
df_hadmid_icd = pd.read_csv(DATA_FILE)
display(df_hadmid_icd.head())
display(df_hadmid_icd.shape)

ICD_PREFIXES = ICD9_CKD_PREFIXES + ICD10_CKD_PREFIXES
COL_PREFIX = "F_icd_code_"
ICD_PREFIXES = [f"{COL_PREFIX}{x}" for x in ICD_PREFIXES]

DIALYSIS_ICD_PREFIXES = [f"{COL_PREFIX}{x}" for x in ["V4511", "Z992"]]


def remove_cols_matching_prefixes(df, prefixes):
    df = df.copy(deep=True)
    df.columns = [x.replace(
        COL_PREFIX, '') if x.startswith(tuple(prefixes)) else x for x in df.columns]
    return df


# if column name start matches any string in ICD_PREFIXES, remove the prefix
df_hadmid_icd = remove_cols_matching_prefixes(
    df_hadmid_icd, ICD_PREFIXES + DIALYSIS_ICD_PREFIXES)

# for the column 'F_gender' in train_data convert all values of M to 0 and values of F to 1
df_hadmid_icd['F_gender'] = df_hadmid_icd['F_gender'].replace({'M': 0, 'F': 1})

# save df to csv
df_hadmid_icd.to_csv('ckd_icd_codes_ckd_status_stages_grouped_hadmid_v2.csv')


/var/folders/65/qxp2fcwj00bff0rdg3hmhyx80000gn/T/ipykernel_24165/1078818734.py:4: DtypeWarning: Columns (29,40,41,67,68,69,70,84,88,104,105,106,124,125,126) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hadmid_icd = pd.read_csv(DATA_FILE)


,subject_id_hadm_id,subject_id,hadm_id,F_age,F_gender,F_max_bmi,ckd_status,race,insurance,language,ckd_stage,F_icd_code_0389,F_icd_code_2449,F_icd_code_25000,F_icd_code_25040,F_icd_code_25060,F_icd_code_2720,F_icd_code_2724,F_icd_code_2749,F_icd_code_2760,F_icd_code_2761,F_icd_code_2762,F_icd_code_27651,F_icd_code_27652,F_icd_code_2767,F_icd_code_2768,F_icd_code_27800,F_icd_code_2809,F_icd_code_2851,F_icd_code_28521,F_icd_code_2859,F_icd_code_2875,F_icd_code_28860,F_icd_code_30000,F_icd_code_3051,F_icd_code_311,F_icd_code_32723,F_icd_code_33829,F_icd_code_3572,F_icd_code_4019,F_icd_code_40390,F_icd_code_40391,F_icd_code_41071,F_icd_code_412,F_icd_code_41400,F_icd_code_41401,F_icd_code_4168,F_icd_code_4240,F_icd_code_4241,F_icd_code_42731,F_icd_code_42789,F_icd_code_4280,F_icd_code_42822,F_icd_code_42823,F_icd_code_42832,F_icd_code_42833,F_icd_code_4439,F_icd_code_4589,F_icd_code_486,F_icd_code_49390,F_icd_code_496,F_icd_code_51881,F_icd_code_53081,F_icd_code_56400,F_icd_code_58381,F_icd_code_5845,F_icd_code_5849,F_icd_code_5853,F_icd_code_5854,F_icd_code_5856,F_icd_code_5859,F_icd_code_5990,F_icd_code_60000,F_icd_code_71590,F_icd_code_73300,F_icd_code_78659,F_icd_code_78791,F_icd_code_78820,F_icd_code_79092,F_icd_code_79902,F_icd_code_99592,F_icd_code_A419,F_icd_code_D509,F_icd_code_D62,F_icd_code_D631,F_icd_code_D649,F_icd_code_D696,F_icd_code_E039,F_icd_code_E1122,F_icd_code_E1165,F_icd_code_E119,F_icd_code_E669,F_icd_code_E785,F_icd_code_E8497,F_icd_code_E860,F_icd_code_E871,F_icd_code_E872,F_icd_code_E875,F_icd_code_E8798,F_icd_code_F329,F_icd_code_F419,F_icd_code_G4733,F_icd_code_G92,F_icd_code_I10,F_icd_code_I120,F_icd_code_I129,F_icd_code_I130,F_icd_code_I214,F_icd_code_I2510,F_icd_code_I252,F_icd_code_I480,F_icd_code_I4891,F_icd_code_I5022,F_icd_code_I5023,F_icd_code_I5032,F_icd_code_I5033,F_icd_code_I959,F_icd_code_J189,F_icd_code_J449,F_icd_code_J9601,F_icd_code_K219,F_icd_code_K5900,F_icd_code_M109,F_icd_code_N179,F_icd_code_N183,F_icd_code_N186,F_icd_code_N189,F_icd_code_N390,F_icd_code_N400,F_icd_code_R0902,F_icd_code_V1251,F_icd_code_V1254,F_icd_code_V1582,F_icd_code_V4501,F_icd_code_V4511,F_icd_code_V4581,F_icd_code_V4582,F_icd_code_V4986,F_icd_code_V5861,F_icd_code_V5866,F_icd_code_V5867,F_icd_code_Y92009,F_icd_code_Y92230,F_icd_code_Y92239,F_icd_code_Y929,F_icd_code_Z515,F_icd_code_Z66,F_icd_code_Z7901,F_icd_code_Z7902,F_icd_code_Z794,F_icd_code_Z8673,F_icd_code_Z87891,F_icd_code_Z951,F_icd_code_Z955,F_icd_code_Z992,F_max_blood_pressure_systolic,F_max_blood_pressure_diastolic
0,10132888_21293774,10132888,21293774,89,M,NaN,True,ASIAN,Other,?,3.0,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN
1,10246901_28798682,10246901,28798682,51,M,29.5,True,WHITE,Other,ENGLISH,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,Fa

(365693, 157)

In [ ]:
display(list(df_hadmid_icd.columns))

# print number of columns that start with "F_"
print(len([x for x in df_hadmid_icd.columns if x.startswith('F_')]))